# 1. Install and import all necessary libraries

In [2]:
# import common packages
import re
import sys
import itertools
import datetime
from tqdm.notebook import tqdm
import pandas_profiling
#from currency.converter import CurrencyConverter
from datetime import datetime

# import visualization packages
import seaborn as sns
import matplotlib.pyplot as plt

# import packages to work with the numeric, tabular data
import numpy as np 
import pandas as pd 

# import ML packagesb
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, PolynomialFeatures
from sklearn.feature_selection import f_regression, mutual_info_regression
from sklearn.model_selection import train_test_split, KFold, RandomizedSearchCV, cross_val_score
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor, ExtraTreesRegressor, AdaBoostRegressor, GradientBoostingRegressor, StackingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.metrics import r2_score

from catboost import CatBoostRegressor
import xgboost as xgb

from hyperopt import tpe, hp, fmin, STATUS_OK,Trials
from hyperopt.pyll.base import scope

import warnings
warnings.filterwarnings("ignore")

In [3]:
!pip freeze > requirements.txt

# 2. Setup

In [4]:
RANDOM_SEED = 42
VERSION    = 4
VAL_SIZE= 0.20   # 20%

In [5]:
cols_to_remove = [] # создаем список столбцов к удалению после создания новых признаков
dropped_columns = [] # создаем список столбцов удаленных
add_columns = [] # создаем список созданных столбцов 
num_cols = [] # создаем список числовых столбцов 
bin_cols = [] # создаем список бинарных столбцов 
cat_cols = [] # создаем список категориальных столбцов 

# 3. Первичный осмотр данных

In [6]:
df_test = pd.read_csv('../input/sf-dst-car-price-prediction/test.csv')
df_train = pd.read_csv('../input/final-car-price-prediction-df-parsed-sep-2021/train_df_full_part1.csv')
df_sub = pd.read_csv('../input/sf-dst-car-price-prediction/sample_submission.csv')

In [7]:
# осматриваем датасеты и пропуски
for i in [df_train, df_test]:
    print(i.info())
    print(i.isna().sum())

видим, что в тренировочном много пропусков, в тестовой не хватает данных в столбцах complectation_dict, equipment_dict и Владение.
кроме того, количество характеристик во всех трех выборках не совпадает, проверяем где разница в столбцах

In [8]:
df_test.columns,df_train.columns

видим, что в тестовой нет показателя цены, довавляем в тестовую дополнительный  столбец
удаляем 3 столбца из трейна

In [9]:
df_train = df_train.drop(['views', 'date_added', 'region'], axis=1)

In [10]:
dropped_columns.extend(['views', 'date_added', 'region'])
dropped_columns

In [11]:
# добавим в тестовый датафрейм столбец с итоговым значением для корректной обработки данных в дальнейшем
df_test['price'] = 0

# отметим тренеровочный и тестовый
df_test['markup'] = 0
df_train['markup'] = 1

объединяем датасеты

In [12]:
data = df_train.append(df_test, sort=False).reset_index(drop=True) # объединяем
data.info()

проверяем на дупликаты

In [13]:
len(data) - len(data.drop_duplicates())

In [14]:
data.drop_duplicates().sort_values("complectation_dict")

дупликаты показывают некорректную информацию по одному sell_id, car_url, complectation_dict могут быть машины с разным цветом, ценой и описанием

In [15]:
# переименуем столбцы
data.rename(columns={'Владельцы': 'owners', 'Владение': 'ownership', 'ПТС': 'vehicle_licence',
       'Привод': 'driving_gear', 'Руль': 'steering_wheel', 'Состояние': 'condition', 'Таможня': 'customs'}, inplace=True)

# 4. EDA

проверяем на пропуски

In [16]:
data.isna().sum()

In [17]:
data.info()

## 4.1. Категориальные признаки

### bodyType

In [18]:
data['bodyType'].value_counts()

проверяем на пропуски, проверяем к какому сету относятся и видим что они все относятся к трейну, можно смело удалять ряд

In [19]:
temp = data[data['bodyType'].isnull()]
temp[temp.markup == 1].count()

In [20]:
data = data.drop(data[(data['bodyType']).isnull()].index)
data.info()

In [21]:
cat_cols.append('bodyType')

In [22]:
def bodytypefunction(s):
    if "внедорожник" in s:
        return "внедорожник"
    elif "седан" in s:
        return "седан"
    elif "универсал" in s:
        return "универсал"
    elif "хэтчбек" in s:
        return "хэтчбек"
    elif "фастбек" in s:
        return "хэтчбек"
    elif "тарга" in s:
        return "кабриолет"
    elif "родстер" in s:
        return "кабриолет"  
    elif "пикап" in s:
        return "пикап" 
    elif "компактвэн" in s:
        return "минивэн" 
    elif "микровэн" in s:
        return "минивэн" 
    elif "купе" in s:
        return "купе" 
    else: return s
data['bodyType'] = data['bodyType'].apply(bodytypefunction)
data['bodyType'].value_counts()

### brand

In [23]:
data['brand'].value_counts()

создаем новую характеристику, аварийность

In [24]:
lstmk = ["daewoo", "smart", "genesis","jaguar", "audi", "mercedes", "infinity", "citroen", "skoda", "lexus"]
def dtp1(a):
    if a.lower() in lstmk:
        return 1
    else:
        return 0
data["accident_poss"] = data["brand"].apply(dtp1)
data["accident_poss"].value_counts()

In [25]:
data[data['brand'].isnull()]


In [26]:
cat_cols.append('brand')
add_columns.append('accident_poss')


### car_url

In [27]:
data.drop(['car_url'], axis=1, inplace=True)

удаляем столбец, т.к. он содержит только линки 

In [28]:
dropped_columns.append('car_url')

### color

In [29]:
data.color.value_counts()

In [30]:
data.color.isnull().count()

In [31]:
cat_cols.append('color')

### complectation_dict

In [32]:
data['complectation_dict']

In [33]:
data = data.drop(["complectation_dict"], axis = 1)

In [34]:
dropped_columns.append('complectation_dict')

### description

In [35]:
data['description']

In [36]:
data["description"] = data["description"].fillna("Нет описания")

In [37]:
cols_to_remove.append('description')

### engineDisplacement

In [38]:
data.engineDisplacement.unique()

In [39]:
def convert_engineDisplacement_to_float(row):
    extracted_value = re.findall('\d\.\d', str(row))
    if extracted_value:
        return float(extracted_value[0])
    return 0

In [40]:
data.engineDisplacement = data.name.apply(convert_engineDisplacement_to_float)
data.engineDisplacement.unique()

In [41]:
num_cols.append('engineDisplacement')

### enginePower

In [42]:
print(data.enginePower.unique())

In [43]:
data['enginePower'].fillna(0)
data['enginePower'] = data['enginePower'].str.replace(' N12', '')
data['enginePower'] = data['enginePower'].astype(int)

In [44]:
data[data['enginePower'].isnull()]

In [45]:
num_cols.append('enginePower')

### equipment_dict

In [46]:
data['equipment_dict'].value_counts()

In [47]:
data.drop(['equipment_dict'], axis=1, inplace=True)

In [48]:
dropped_columns.append('equipment_dict')

### fuelType

In [49]:
data['fuelType'].value_counts()

In [50]:
data['fuelType'].isnull().count()

In [51]:
cat_cols.append("fuelType")

### image

In [52]:
data.drop(['image'], axis=1, inplace=True)

удаляем ненужный столбец

In [53]:
dropped_columns.append('image')

### model_info

In [54]:
data.model_info.value_counts()

большую часть составляет информация об марке, классе, эти данные получаем из других столбцов, поэтому можно удалить

In [55]:
data.drop(['model_info'], axis=1, inplace=True)

In [56]:
dropped_columns.append('model_info')

### model_name

In [57]:
cols_to_remove.append('model_name')

будем использовать в дальнейшем создании признаков

### name

In [58]:
data['name'].unique()

In [59]:
cols_to_remove.append('name')

будем использовать в дальнейшем создании признаков

### priceCurrency

все суммы указаныв рублях, код валюты можно удалить

In [60]:
data = data.drop(['priceCurrency'], axis=1)

In [61]:
dropped_columns.append('priceCurrency')

### super_gen

In [62]:
data.drop(['super_gen'], axis=1, inplace=True)

In [63]:
dropped_columns.append('super_gen')

### vehicleConfiguration

In [64]:
data['vehicleConfiguration'].value_counts()

In [65]:
data.info()

In [66]:
cat_cols.append('vehicleConfiguration')

### vehicleTransmission

In [67]:
data.vehicleTransmission.unique()

In [68]:
data['vehicleTransmission'].value_counts()

In [69]:
cat_cols.append('vehicleTransmission')

### vendor

In [70]:
print(data.vendor.value_counts())
print(data.vendor.unique())
data.vendor.isnull().value_counts()

In [71]:
data[data.vendor == "AMERICAN"]

VOLKSWAGEN европейская машина, поэтому меняем на европейскую

In [72]:
vendor_dict = {k:v for v,k in data.groupby(['vendor', 'brand']).name.count().index}
print(vendor_dict)
data['vendor'] = data.brand.map(vendor_dict)
data[data.vendor.isnull()]

In [73]:
print(data.vendor.value_counts())
print(data.vendor.unique())
data.vendor.isnull().value_counts()

In [74]:
cat_cols.append('vendor')

### owners

In [75]:
print(data.owners.unique())
data['owners'].value_counts()

In [76]:
def convert_owners_to_float(value):
    if isinstance(value, str):
        return float(value.replace('\xa0', ' ').split()[0])
    return value

In [77]:
data.owners = data.owners.apply(convert_owners_to_float)
data.owners.unique()

In [78]:
data.loc[(data.owners.isna()) & (data.mileage == 0.0), 'owners'] = 0

In [79]:
print(data[data.owners.isnull()].groupby('productionDate').modelDate.count())
data[data.owners.isnull()].groupby('productionDate').median()
data.loc[data.owners.isnull(), ['mileage', 'owners']] = 0


In [80]:
cat_cols.append('owners')

### ownership

In [81]:
data.drop(['ownership'], axis=1, inplace=True)

In [82]:
dropped_columns.append('ownership')

### vehicle_licence

In [83]:
data.vehicle_licence.value_counts()

In [84]:
data.loc[(data.vehicle_licence.isna()) & (data.mileage == 0.0), 'vehicle_licence'] = 'Оригинал'
data.loc[(data.vehicle_licence.isna()) & (data.mileage != 0.0), 'vehicle_licence'] = 'Дубликат'

In [85]:
data.vehicle_licence.unique()

In [86]:
cat_cols.append('vehicle_licence')

### driving_gear

In [87]:
data['driving_gear'].value_counts()

In [88]:
driving_gear_df = data.groupby(['brand', 'model_name', 'driving_gear']).bodyType.count().reset_index().drop_duplicates(['brand', 'model_name']).drop(columns=['bodyType'])
driving_gear_df

In [89]:
def fill_driving_gear(brand, model_name, df):
    sliced_df = df[(df.brand == brand) & (df.model_name == model_name)]
    try:
        return sliced_df.driving_gear.values[0]
    except:
         return 'полный' 

In [90]:
data.loc[data.driving_gear.isna(), 'driving_gear'] = data[data.driving_gear.isna()].apply(
    lambda row: fill_driving_gear(row.brand, row.model_name, driving_gear_df), axis=1)

In [91]:
data.info()

In [92]:
cat_cols.append('driving_gear')

### steering_wheel

In [93]:
data['steering_wheel'].value_counts()

In [94]:
data.steering_wheel.unique()

In [95]:
data.steering_wheel.value_counts()
right_wheel_models = data[data.steering_wheel == 'Правый'].groupby(['brand', 'model_name']).bodyType.count() 
right_wheel_models = right_wheel_models.reset_index().sort_values('bodyType', ascending=False)
right_wheel_dict = right_wheel_models[right_wheel_models.bodyType > 1].groupby('brand').agg({'model_name': lambda x: x.tolist()}).to_dict()['model_name']
right_wheel_dict

In [96]:
def fill_steering_wheel(brand, model, right_wheel_dict):
    try:
        if model in right_wheel_dict[brand]:
            return 'Правый'
        else:
            return 'Левый'
    except:
        return 'Левый'

In [97]:
data.loc[data.steering_wheel.isna(), 'steering_wheel'] = data[data.steering_wheel.isna()].apply(
    lambda row: fill_steering_wheel(row.brand, row.model_name, right_wheel_dict), axis=1)

In [98]:
data.steering_wheel.value_counts()

In [99]:
cat_cols.append('steering_wheel')

### condition

In [100]:
data.condition.value_counts()


In [101]:
data.condition = data.condition.apply(lambda x: 1 if x == 'Не требует ремонта' else 0)

In [102]:
bin_cols.append('condition')

### customs

In [103]:
print(data.customs.unique())

In [104]:
data.customs = data.customs.apply(lambda x: 1 if x == 'Растаможен' or x == True else 0)
print(data.customs.unique())

In [105]:
bin_cols.append('customs')

## 4.2. Postcategorical (итоги)

создан один числовой признак, удалены 8 лишних признаков
заполнены пробелы во всех категориальных  признаках кроме model_name, его планируем использовать ниже и удалять

In [106]:
data.info(), data[data.markup == 1].info(), data[data.markup == 0].info()

In [107]:
print("список столбцов к удалению после создания новых признаков: ", cols_to_remove)
print("список столбцов удаленных: ",dropped_columns)
print("список созданных столбцов: ",add_columns) 
print("список числовых столбцов: ",num_cols) 
print("список бинарных столбцов: ", bin_cols) 
print("список категориальных столбцов : ", cat_cols)

## 4.3. Числовые признаки

### mileage

In [108]:
data['mileage'].value_counts()

In [109]:
num_cols.append('mileage')

### modelDate

In [110]:
data['modelDate'].value_counts()

** используем позже для анализа сроков пользования

In [111]:
cols_to_remove.append('modelDate')

### numberOfDoors

In [112]:
data.numberOfDoors.value_counts()

In [113]:
cat_cols.append('numberOfDoors')

### parsing_unixtime

In [114]:
data['parsing_unixtime'].value_counts()

In [115]:
data['parsing_date'] = pd.to_datetime(data.parsing_unixtime, unit='s')
data['parsing_date']

In [116]:
data['parsing_date'] = data['parsing_date'].dt.to_period('M')
data['parsing_date'].value_counts()     

приводим все даты парсинга к одной дате

In [117]:
data.drop(['parsing_unixtime'], axis=1, inplace=True)

In [118]:
dropped_columns.append('parsing_unixtime')
add_columns.append('parsing_date')

### productionDate

признак будет использван позже для создания новых признаков

In [119]:
num_cols.append('productionDate')

### sell_id

можно удалять

In [120]:
data.drop(['sell_id'], axis=1, inplace=True)

In [121]:
dropped_columns.append('sell_id')

### price

через сайт https://xn----ctbjnaatncev9av3a8f8b.xn--p1ai/%D1%82%D0%B0%D0%B1%D0%BB%D0%B8%D1%86%D1%8B-%D0%B8%D0%BD%D1%84%D0%BB%D1%8F%D1%86%D0%B8%D0%B8 узнаем цену рубля в определенный месяц и приводим цены к цене на март 2022

In [122]:
data.price

In [123]:
data["new_price2020-10"] = data[data['parsing_date'] == ((pd.to_datetime('2020-10')).to_period('M'))].price  * 1.00134 * 1.0839 * 1.00217
data["new_price2020-10"] = data["new_price2020-10"].fillna(0)
data["new_price2021-09"] = data[data['parsing_date'] == ((pd.to_datetime('2021-09')).to_period('M'))].price  * 1.0017 * 1.00217
data["new_price2021-09"] = data["new_price2021-09"].fillna(0)
data["new_price2021-10"] = data[data['parsing_date'] == ((pd.to_datetime('2021-10')).to_period('M'))].price  * 1.00098 * 1.00217
data["new_price2021-10"] = data["new_price2021-10"].fillna(0)

In [124]:
data["price"] = data["price"] - data["price"] + data["new_price2021-10"] + data["new_price2021-09"] + data["new_price2020-10"]

In [125]:
data.drop(["new_price2021-10","new_price2021-09","new_price2020-10"], axis=1, inplace=True)

In [126]:
dropped_columns.extend(["new_price2021-10","new_price2021-09","new_price2020-10"])

In [127]:
data.info()

In [128]:
data[data.markup == 1].price

In [129]:
data[data.markup == 1].price.isna().sum(), data[data.markup == 0].price.isna().sum()


в трейне есть пропуски на целевой переменной, удаляем их, не будем моделиривать на спрогнозированных данных

In [130]:
data = data.drop(data[(data['markup'] == 1) & (data['price'].isnull())].index)

In [131]:
data.info(), data[data.markup == 1].info(), data[data.markup == 0].info()

In [132]:
data.info()

### markup

различитель , не относим ни к  одной из групп

## 4.4. Postnumerical (итоги)

In [133]:
data.info(), data[data.markup == 1].info(), data[data.markup == 0].info()

In [134]:
print("список столбцов к удалению после создания новых признаков: ", cols_to_remove)
print("список столбцов удаленных: ",dropped_columns)
print("список созданных столбцов: ",add_columns) 
print("список числовых столбцов: ",num_cols) 
print("список бинарных столбцов: ", bin_cols) 
print("список категориальных столбцов : ", cat_cols)

осталось 2 столбца с пропусками, model_name и price. осталось обработать их и проанализиртовать новые добавленные столбцы

# 5. Обработка на основе новых созданных числовых признаков

### accident_poss

In [135]:
data.accident_poss.value_counts()

### parsing_date

In [136]:
data.parsing_date.value_counts()

### model_name

In [137]:
def fill_model_name_with_name(all_model_names, name):
    name = name.lower()
    if isinstance(name, str):
        if "RS Q8".lower() in name:
            return 'Q8'
        elif "QX55".lower() in name:
            return "QX55"
        try:
            value = name.split()
        except:
            return None
        if value[1] in all_model_names:
            return value[1]
        try:
            joined_value = "_".join([value[1], value[2]])
        except IndexError:
            return None
        if joined_value in all_model_names:
            return f"{value[1]}_{value[2]}"
        elif joined_value.replace('_', ' ') in all_model_names:
            return f"{value[1]} {value[2]}"
    return None

In [138]:
data[data.model_name.isna()].parsing_date.unique()
data.model_name = data.model_name.str.lower().str.replace('_', ' ')
data.model_name.isna().sum()

In [139]:
available_model_names = list(data.model_name.str.lower().unique())
data.loc[data.model_name.isna(), 'model_name'] = data[data.model_name.isna()].name.apply(lambda x: fill_model_name_with_name(available_model_names, x))

In [140]:
data.loc[data.model_name.isna(), 'model_name'].count()

In [141]:
data.info()

In [142]:
# pandas_profiling.ProfileReport(data)

In [143]:
cols_to_remove.extend(['vehicleConfiguration'])
cat_cols.remove('vehicleConfiguration')

In [144]:
data.isna().sum(axis=0) * 100 / data.shape[0] 

In [145]:
# missing values for the train dataset
sns.heatmap(data[data.markup == 1].isna(), cbar=False);

In [146]:
# missing values for the test dataset
sns.heatmap(data[data.markup == 0].isna(), cbar=False);

In [147]:
# for col in ['enginePower', 'numberOfDoors', 'productionDate', 'owners', 'modelDate']:
#     combined_df[col] = combined_df[col].astype(int)

# 6. Поиск выбросов

In [148]:
print("список столбцов к удалению после создания новых признаков: ", cols_to_remove)
print("список столбцов удаленных: ",dropped_columns)
print("список созданных столбцов: ",add_columns) 
print("список числовых столбцов: ",num_cols) 
print("список бинарных столбцов: ", bin_cols) 
print("список категориальных столбцов : ", cat_cols)

In [149]:
data.info()

## 6.1. Определяем тип столбцов

In [150]:
cat_cols = ['bodyType', 'brand', 'color', 'fuelType', 'model_name', 'name', 'driving_gear', 'owners', 'numberOfDoors']
num_cols = ['engineDisplacement', 'enginePower', 'mileage', 'modelDate', 'productionDate']
bin_cols = ['condition', 'customs', 'steering_wheel', 'vehicleTransmission', 'vendor', 'vehicle_licence', 'accident_poss']
help_cols = ['markup', 'parsing_date']
target_cols = ['price']

all_cols = cat_cols + num_cols + bin_cols + help_cols + target_cols
len(all_cols)

# 6.2. Числовые столбцы

In [151]:
sns.pairplot(data[num_cols]);

Видна сильная корреляция двух пар: ('modelDate', 'productionDate') и ('engineDisplacement', 'enginePower'). по другим линейной зависимости нет, есть группирование в квадрантах

In [152]:
data[num_cols].describe()

попробуем нормализовать данные, чтоб увидеть более четкую зависимость

In [153]:
def vis_num_feature( data, column, target_column, query_for_slicing):
    plt.style.use('seaborn-paper')
    fig, ax = plt.subplots(2, 2, figsize=(15, 9))
    data[column].plot.hist(ax=ax[0][0])
    ax[0][0].set_title(column)
    sns.boxplot(data=data, y=column, ax=ax[0][1], orient='v')
    #sns.scatterplot(data=data.query(query_for_slicing), x=column, y=target_column, ax=ax[0][2])
    np.log2(data[column] + 1).plot.hist(ax=ax[1][0])
    ax[1][0].set_title(f'log2 transformed {column}')
    sns.boxplot(y=np.log2(data[column]), ax=ax[1][1], orient='v')
    plt.show()

In [154]:
# калькуляция выбросов
def calculate_stat_outliers(data_initial, column, log):
    data = data_initial.copy()
    if log:
        data[column] = np.log2(data[column] + 1)
    q1 = data[column].quantile(0.25)
    q3 = data[column].quantile(0.75)
    IQR = q3 - q1
    mask25 = q1 - IQR * 1.5                   
    mask75 = q3 + IQR * 1.5

    values = {}
    values['borders'] = mask25, mask75
    values['# outliers'] = data[(data[column] < mask25)].shape[0], data[data[column] > mask75].shape[0]

    return pd.DataFrame.from_dict(data=values, orient='index', columns=['left', 'right'])

In [155]:
for each in num_cols:
    display(vis_num_feature(data, each, 'price', 'markup == 1'))
    display(calculate_stat_outliers(data, each, log=True))
    print('\n' + '-' * 10 + '\n')

Логарифмирование помогло с enginePower, на остальных характеристиках не удалось улучшить ситуацию. Так же заметно большое количество  выбросов

In [156]:
plt.figure(figsize=(15, 8));
sns.heatmap(data[data.markup == 1][num_cols + ['price']].corr(), vmin=-1, vmax=1, annot=True, cmap='vlag');

 На modelDate и productionDate сильная корреляция Пирсона 0.97. Оставляем modelDate, т.к. productionDate коррелирует с другим показателем

 На productionDate и mileage сильная корреляция Пирсона  -0.81. Оба показателя одинаково важны, но оставляем mileage
 
 engineDisplacement и enginePower графически показывали сильную линейную связь, но на корреляции показывают коэффициэнт 0.0094, надо будет перепроверить

In [157]:
cols_to_remove.extend(['productionDate'])

In [158]:
data.query('markup == 1').enginePower.describe(), data.query('markup == 0').enginePower.describe()

In [159]:
data.query('markup == 1').mileage.describe(), data.query('markup == 0').mileage.describe()

In [160]:
plt.figure(figsize=(15, 6))
for year in data.modelDate.value_counts().index[:5]:
    data[data.modelDate == year].mileage.hist(bins=50, alpha=0.5)
plt.xlim(0, 400000)
plt.ylim(0, 4000)

Распределение пробега по годам выпуска модели различается

In [161]:
plt.figure(figsize=(15, 6))
sns.scatterplot(data=data[data['markup'] == 1], x='mileage', y="price")

видим распределение вдоль осей, чем больше пробег, тем меньше цена

In [162]:
data.query('markup == 1').modelDate.describe(), data.query('markup == 0').modelDate.describe()

In [163]:
data[data.modelDate < 1960]

есть 32 машины старше 1960 года

In [164]:
plt.figure(figsize=(25, 6))
sns.scatterplot(data=data[data['markup'] == 1], x='modelDate', y="price")

## 6.3. Бинарные и категориальные столбцы

In [165]:
def show_boxplot(data, column, target_column):
    fig, ax = plt.subplots(figsize = (14, 4))
    sns.boxplot(x=column, y=target_column, 
                data=data.loc[data.loc[:, column].isin(data.loc[:, column].value_counts().index)],
                ax=ax)
    plt.xticks(rotation=45)
    ax.set_title('Boxplot for ' + column)
    plt.show()

In [166]:
for col in bin_cols + cat_cols:
    if col not in ['model_name', 'name']: # these columns have too many categories, the plots don't show any useful information for them. Create new features!
        fig, ax = plt.subplots(figsize=(15, 4), ncols=2, nrows=1)
        ax[0].set_title(f'TRAIN: # observations in {col} column.', fontdict={'fontsize': 14})
        data[data.markup == 1][col].value_counts(normalize=True).plot(kind='bar', ax=ax[0])
        ax[1].set_title(f'TEST: # observations in {col} column.', fontdict={'fontsize': 14})
        data[data.markup == 0][col].value_counts(normalize=True).plot(kind='bar', ax=ax[1])
        # to visualize the boxplots for the price that has a lot of outliers we'll use 90% quantile for the price
        show_boxplot(data=data[data.price <= data.price.quantile(0.9)], column=col, target_column='price')
        print('_' * 150)
        print('\n')
        plt.show()

несбалансированные condition, customs

слабосблаансированные steeting_wheel, vehicleTransmission, vehicle_licence, bodyType, brand, color, fuelType, numberOfDoors, model, model_name

сбалансированные vendor, driving_gear, owners

In [167]:
cols_to_remove.extend(['condition', 'customs'])

# 7. Анализ целевой переменной

In [168]:
data.query('markup == 1').price.hist();
plt.title('The target variable distribution', fontdict={'fontsize': 14});
plt.xlabel('price, RUB * 10^7');

целевая смещена вправо, логарифмируем

In [169]:
np.log2(data.query('markup == 1').price).hist();
plt.title('Логарифмированная цена', fontdict={'fontsize': 14});

добавляем новый столбец

In [170]:
data['price_log2'] = np.log2(data.price + 1)
# combined_df['price_log2'].replace([np.inf, -np.inf], 0, inplace=True)

In [171]:
data.parsing_date.unique()

In [172]:
data.parsing_date = data.parsing_date.dt.strftime("%Y-%m-%d")
data.parsing_date.unique()

In [173]:
# conda install -c auto converter

In [174]:
# c = CurrencyConverter(fallback_on_missing_rate=True)

# converter_dict = {}
# for each in data.parsing_date.unique():
    # no data from September 2021
#    year, month, day = [int(value) for value in each.split('-')]
#    try:
#        converter_dict[each] = c.convert(1, 'RUB', 'EUR', date=datetime(year, month, day))
#    except:
#        print(each)
        
#converter_dict['2021-09-25'] = 0.01172
#converter_dict['2021-09-26'] = 0.01174
#converter_dict['2021-09-27'] = 0.01177
#converter_dict['2021-09-28'] = 0.01180
#converter_dict['2021-09-29'] = 0.01181
#converter_dict['2021-09-30'] = 0.01186
#converter_dict['2021-10-01'] = 0.01180

In [175]:
# data['price_EURO'] = data.apply(lambda x: x.price * converter_dict[x.parsing_date] if x.price != 0 else x.price, axis=1)

# 8. Feature engineering

### numerical columns
- `mileage_per_year`: using the `productionDate` and `mileage` columns to extract the information how many km the auto drove per year;
- `age_years`: how old is the auto in years;
- `time_bw_model_product`: the difference in years between the release of the model and her production;
- `descr_words_count`: the count of words in the description of the auto;

### binary columns
- `rarity`: whether the auto was produced before 1940;
- `older_3y`: whether the auto is older than 3 years;
- `older_5y`: whether the auto is older than 5 years;
- `sport`: weather the auto is a sport car; #was removed from the analysis
- `dealer`: whether the auto dealer published an ad; #was removed from the analysis
- `top2_bodyType`: whether the auto has ‘внедорожник’ and ‘седан’ body type;
- `rate_bodyType` : whether the auto has the rare body type: 'микровэн', 'седан-хардтоп', 'лимузин', 'тарга', 'фастбек';
- `top5_colors`:  whether the auto has top5 colors: черный, белый, серый, синий, серебристый;
- `rare_colors`: whether the auto has a rare color: 'фиолетовый', 'пурпурный', 'золотистый', 'оранжевый', 'жёлтый', 'розовый';
- `top2_door_num`:  whether the auto has 4 or 5 doors;

In [176]:
# new numerical columns
data['mileage_per_year'] = data.productionDate / data.mileage
data['mileage_per_year'].replace([np.inf, -np.inf], 0, inplace=True)
data['age_year'] = 2021 - data.productionDate
data['age_year'].replace([np.inf, -np.inf], 0, inplace=True)
data['time_bw_model_product'] = data.productionDate - data.modelDate
data.loc[data.time_bw_model_product < 0, 'time_bw_model_product'] = 0
data['descr_words_count'] = data.description.apply(lambda x: len(x.split()) if isinstance(x, str) else 0)
data["состояние"] = data['description'].apply(lambda x: 1 if "хорош" in x.lower() or "отличн" in x.lower() else 0)
data['descr_words_count'].replace([np.inf, -np.inf], 0, inplace=True)

num_cols_added = ['mileage_per_year', 'age_year', 'time_bw_model_product', 'descr_words_count']

In [177]:
# data = data.drop(["description"], axis = 1)

In [178]:
# new binary columns
data['rarity'] = data.productionDate.apply(lambda x: 1 if x < 1960 else 0)
data['older_3y'] = data.productionDate.apply(lambda x: 1 if x < 2021 - 3 else 0)
data['older_5y'] = data.productionDate.apply(lambda x: 1 if x < 2021 - 5 else 0)

data['top2_bodyType'] = data.bodyType.apply(lambda x: 1 if x in ['внедорожник', 'седан'] else 0)
data['rare_bodyType'] = data.bodyType.apply(lambda x: 1 if x in ['микровэн', 'седан-хардтоп', 'лимузин', 'тарга', 'фастбек'] else 0)
data['top5_colors'] = data.color.apply(lambda x: 1 if x in ['черный', 'белый', 'серый', 'синий', 'серебристый'] else 0)
data['rare_colors'] = data.color.apply(lambda x: 1 if x in ['фиолетовый', 'пурпурный', 'золотистый', 'оранжевый', 'жёлтый', 'розовый'] else 0)
data['top2_door_numb'] = data.numberOfDoors.apply(lambda x: 1 if x in [4, 5] else 0)

bin_cols_added = ['rarity', 'older_3y', 'older_5y', 'top2_bodyType', 'rare_bodyType', 'top5_colors', 'rare_colors', 'top2_door_numb','accident_poss']

## 8.1. Analysis

In [179]:
for each in num_cols_added:
    display(vis_num_feature(data, each, 'price', 'markup == 1'))
    display(calculate_stat_outliers(data, each, log=True))
    print('\n' + '-' * 10 + '\n')

In [180]:
data['age_year_log2'] = np.log2(data.age_year+1)
# combined_df['age_year_log2'].replace([np.inf, -np.inf], 0, inplace=True)
data['time_bw_model_product_log2'] = np.log2(data.time_bw_model_product+1)
# combined_df['descr_words_count_log2'] = np.log2(combined_df.descr_words_count+1)

data['mileage_per_year_log2'] = np.log2(data.mileage_per_year+1)
# combined_df['mileage_per_year_log2'].replace([np.inf, -np.inf], 0, inplace=True)

In [181]:
num_cols_added.remove('age_year')
num_cols_added.remove('mileage_per_year')
num_cols_added.remove('time_bw_model_product')

In [182]:
plt.figure(figsize=(15, 8));
sns.heatmap(data[data.markup == 1][num_cols_added + ['time_bw_model_product_log2',
    'age_year_log2', 'mileage_per_year_log2', 
    'price']].corr(), vmin=-1, vmax=1, annot=True, cmap='vlag');

Оставляем прологарифмированные данные

In [183]:
num_cols_added.extend(['time_bw_model_product_log2', 'age_year_log2', 'mileage_per_year_log2'])

In [184]:
for col in bin_cols_added:
    fig, ax = plt.subplots(figsize=(15, 4), ncols=2, nrows=1)
    ax[0].set_title(f'TRAIN: # observations in {col} column.', fontdict={'fontsize': 14})
    data[data.markup == 1][col].value_counts(normalize=True).plot(kind='bar', ax=ax[0])
    ax[1].set_title(f'TEST: # observations in {col} column.', fontdict={'fontsize': 14})
    data[data.markup == 0][col].value_counts(normalize=True).plot(kind='bar', ax=ax[1])
    # to visualize the boxplots for the price that has a lot of outliers we'll use 90% quantile for the price
    show_boxplot(data=data[data.price <= data.price.quantile(0.9)], column=col, target_column='price')
    print('_' * 150)
    print('\n')
    plt.show()

Based on the plots we could conclude that all variable can be useful for the model.

## 8.2. Construct final lists of all columns

In [185]:
cols_to_remove.remove('modelDate')
cols_to_remove

In [186]:
data.drop(cols_to_remove, axis=1, inplace=True)

In [187]:
num_cols.extend(num_cols_added)
bin_cols.extend(bin_cols_added)

In [188]:
num_cols, bin_cols, cat_cols

In [189]:
num_cols.remove('age_year_log2')

In [190]:
data['enginePower_log2'] = np.log2(data.enginePower+1)
data['enginePower_log2'].replace([np.inf, -np.inf], 0, inplace=True)
data['mileage_log2'] = np.log2(data.mileage+1)
data['mileage_log2'].replace([np.inf, -np.inf], 0, inplace=True)

In [191]:
num_cols.remove('productionDate')

In [192]:
data.drop(['mileage_log2', 'age_year_log2'], axis=1, inplace=True)

In [193]:
plt.figure(figsize=(15, 8));
sns.heatmap(data[data.markup == 1][num_cols + ['price']].corr(), vmin=-1, vmax=1, annot=True, cmap='vlag');

In [194]:
data.dropna(inplace=True)

In [195]:
imp_num = pd.Series(f_regression(data[data.markup == 1][num_cols], data[data.markup == 1]['price'])[0], index = num_cols)
imp_num.sort_values(inplace = True)
imp_num.plot(kind = 'barh')
plt.xlabel('f-statistic value');

In [196]:
for colum in ['steering_wheel', 'vehicleTransmission', 'vendor', 'vehicle_licence']:
    data[colum] = data[colum].astype('category').cat.codes

cols_to_encode = list(set(data.columns) & set(cat_cols))
for colum in cols_to_encode:
    data[colum] = data[colum].astype('category').cat.codes

In [197]:
# let's look at the importance of the categorical and binary columns
imp_cat = pd.Series(
    mutual_info_regression(
        data[data.markup == 1][list(set(data.columns) & set(cat_cols+bin_cols))], 
        data[data.markup == 1]['price'], 
        discrete_features=True), index=list(set(data.columns) & set(cat_cols+bin_cols))
)
imp_cat.sort_values(inplace=True)
imp_cat.plot(kind='barh', title='The importance of the categorical and binary columns.')
plt.show()

The top5 most important features are: `vehicleTransmission`, `brand`, `color`, `owners` and `bodyType`.

# 9. ML

In [198]:
data.info()

In [199]:


X = data.query('markup == 1').drop(['price', 'markup', 'parsing_date', 'price_log2', 'price'
                                         ], axis=1)

y = data.query('markup == 1').price

## 9.1. Разбиваем выборку

In [200]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)

In [201]:
tmp_train = X_train.copy()
tmp_train['price'] = y_train


In [202]:
tmp_train.info()


In [203]:
def mape(y_true, y_pred):

    return np.mean(np.abs((y_pred-y_true)/y_true))

## 9.2.Наивная модель

In [204]:
# predict = X_test['mileage'].map(tmp_train.groupby('mileage')['price'].median())
# print(f"Точность наивной модели по MAPE: {(mape(y_test, predict.values))*100:0.2f}%.")

Точность наивной модели по MAPE: 20.07%.

## 9.3. Линейная регрессия

In [205]:
# LR = LinearRegression()
# LR.fit(X_train, y_train)
# y_pred = LR.predict(X_test)
# print("MSE по ЛР:", mean_squared_error(y_test, y_pred))
# print("R2 по ЛР:", r2_score(y_test, y_pred))
# print(f'MAPE по ЛР: {mape(y_test, y_pred)*100}%')

MAPE по ЛР: 70.03186281288252%

## 9.4. CatBoost

### Base Traget

In [206]:
# cat_boost = CatBoostRegressor(iterations = 5000,
#                           random_seed = RANDOM_SEED,
#                           eval_metric='MAPE',
#                           custom_metric=['R2', 'MAE'],
#                           silent=True,
#                          )
# 
# skf = KFold(n_splits=5, random_state=RANDOM_SEED)
# cat_boost_mape_values = []
# 
# for train_index, test_index in skf.split(X, y):
#     print("TRAIN:", train_index, "TEST:", test_index)
#     X_train, X_test = X[X.index.isin(train_index)], X[X.index.isin(test_index)]
#     y_train, y_test = y[y.index.isin(train_index)], y[y.index.isin(test_index)]
#     
# 
#     cat_boost.fit(X_train, y_train,
#          eval_set=(X_test, y_test),
#          verbose_eval=0,
#          use_best_model=True,
#          )
# 
#     y_pred = cat_boost.predict(X_test)
# 
#     print("MSE по CatBoost:", mean_squared_error(y_test, y_pred))
#     print("R2 по CatBoost:", r2_score(y_test, y_pred))
#     cat_boost_mape_value = mape(y_test, y_pred)
#     cat_boost_mape_values.append(cat_boost_mape_value)
#     print(f'MAPE по CatBoost: {mape(y_test, y_pred)*100}%')

# print(f"Среднее MAPE по CatBoost на выборке из 5: {(np.mean(cat_boost_mape_values) * 100):0.2f}%.")

Среднее MAPE по CatBoost на выборке из 5: 18.88%

### Log Traget

In [207]:
# cat_boost_log = CatBoostRegressor(iterations = 5000,
#                           random_seed = RANDOM_SEED,
#                           eval_metric='MAPE',
#                           custom_metric=['R2', 'MAE'],
#                           silent=True,
#                          )
# 
# skf = KFold(n_splits=5, random_state=RANDOM_SEED)
# cat_boost_log_mape_values = []
# 
# for train_index, test_index in skf.split(X, y):
#     print("TRAIN:", train_index, "TEST:", test_index)
#     X_train, X_test = X[X.index.isin(train_index)], X[X.index.isin(test_index)]
#     y_train, y_test = y[y.index.isin(train_index)], y[y.index.isin(test_index)]
#     
# 
#     cat_boost_log.fit(X_train, np.log(y_train),
#          eval_set=(X_test, np.log(y_test)),
#          verbose_eval=0,
#          use_best_model=True,
#          )
# 
#     y_pred = np.exp(cat_boost_log.predict(X_test))
# 
#     print("MSE по CatBoost:", mean_squared_error(y_test, y_pred))
#     print("R2 по CatBoost:", r2_score(y_test, y_pred))
#     cat_boost_log_mape_value = mape(y_test, y_pred)
#     cat_boost_log_mape_values.append(cat_boost_log_mape_value)
#     print(f'MAPE по CatBoost: {mape(y_test, y_pred)*100}%')
# 
# print(f"Среднее MAPE по CatBoost на выборке из 5 при стандартизированной целевой: {(np.mean(cat_boost_log_mape_values) * 100):0.2f}%.")

Среднее MAPE по CatBoost на выборке из 5 при стандартизированной целевой: 14.24%.

## 9.5. Random Forest

Let's try with the basic RF model with default hyperparameters.

In [208]:
# without log-transformation of the target variable
# rf = RandomForestRegressor(random_state=RANDOM_SEED, n_jobs=-1, verbose=1)
# rf.fit(X_train, y_train)
# predict_rf = rf.predict(X_test)
# 
# print(f"MAPE по RandomForest: {(mape(y_test, predict_rf) * 100):0.2f}%.")
# 
# 
# rf_log = RandomForestRegressor(random_state=RANDOM_SEED, n_jobs=-1, verbose=1)
# rf_log.fit(X_train, np.log(y_train))
# predict_rf_log = np.exp(rf_log.predict(X_test))
# 
# print(f"MAPE по RandomForest при стандартизированной целевой: {(mape(y_test, predict_rf_log) * 100):0.2f}%.")

MAPE по RandomForest: 12.31%.
MAPE по RandomForest при стандартизированной целевой: 11.65%.

берем среднее по 5 выборкам при стандартизированной целевой

In [209]:
# model = RandomForestRegressor(random_state=RANDOM_SEED, n_jobs=-1, verbose=1)
# 
# skf = KFold(n_splits=5, random_state=RANDOM_SEED)
# mape_values = []
# 
# for train_index, test_index in skf.split(X, y):
#     print("TRAIN:", train_index, "TEST:", test_index)
#     X_train, X_test = X[X.index.isin(train_index)], X[X.index.isin(test_index)]
#     y_train, y_test = y[y.index.isin(train_index)], y[y.index.isin(test_index)]
#     
#     model.fit(X_train, np.log(y_train))
# 
#     y_pred = np.exp(model.predict(X_test))
# 
#     print("MSE по RandomForest:", mean_squared_error(y_test, y_pred))
#     print("R2 по RandomForest:", r2_score(y_test, y_pred))
#     mape_value = mape(y_test, y_pred)
#     mape_values.append(mape_value)
#     print("MAPE по RandomForest:", mape_value)
# 
# print(f"Среднее MAPE по RandomForest на выборке из 5 при стандартизированной целевой: {(np.mean(mape_values) * 100):0.2f}%.")

Среднее MAPE по RandomForest на выборке из 5 при стандартизированной целевой: 13.66%.

тестируем гиперхарактеры и пеоверяем на новых данных модель

In [210]:
# hp.uniform('n_estimators',100,500),
# hp.choice("n_estimators", [int(x) for x in np.linspace(200, 1000, num = 17)])

# def objective(params):
#     model=RandomForestRegressor(
#         n_estimators=int(params['n_estimators']),
#         max_depth=int(params['max_depth']),
#         min_samples_leaf=int(params['min_samples_leaf']),
#         min_samples_split=int(params['min_samples_split']),
#         bootstrap=params['bootstrap'],
#         max_features=params['max_features'],
#         random_state=RANDOM_SEED,
#         n_jobs=-1
#     )
#     model.fit(X_train, np.log(y_train))
#     pred=model.predict(X_test)
#     score=mape(y_test,np.exp(pred))
#     return score

# def optimize(trial):
#     params={
#         'n_estimators': hp.uniform('n_estimators',100,500),
#         'max_features': hp.choice("max_features", ['auto', 'sqrt']),
#         'max_depth': hp.uniform('max_depth',5,15),
#         'min_samples_split': hp.uniform('min_samples_split',2,10),
#         'min_samples_leaf': hp.uniform('min_samples_leaf',1,5),
#         'bootstrap': hp.choice("bootstrap", [True, False])
#     }
#     best=fmin(fn=objective, space=params, algo=tpe.suggest, trials=trial, max_evals=100, rstate=np.random.RandomState(RANDOM_SEED))
#     return best

# trial=Trials()
# best=optimize(trial)
# print(best)

100%|██████████| 100/100 [23:02<00:00, 13.83s/trial, best loss: 0.1129756615602328]  
{'bootstrap': 0, 'max_depth': 14.181147066930533, 'max_features': 1, 'min_samples_leaf': 1.711876393930417, 'min_samples_split': 4.907320931834125, 'n_estimators': 152.01315530496686}

In [211]:
# with log-transformation of the target variable
# rf_opt_log = RandomForestRegressor(random_state=RANDOM_SEED, 
#                                    n_jobs=-1, 
#                                    verbose=1, 
#                                    n_estimators = 152, 
#                                    min_samples_split = 5, 
#                                    min_samples_leaf = 2, 
#                                    max_features = 1, 
#                                    max_depth = 14, 
#                                    bootstrap = 0)
# 
# rf_opt_mape_values = []
# skf = KFold(n_splits=4, random_state=RANDOM_SEED)
# 
# for train_index, test_index in skf.split(X, y):
#     print("TRAIN:", train_index, "TEST:", test_index)
#     X_train, X_test = X[X.index.isin(train_index)], X[X.index.isin(test_index)]
#     y_train, y_test = y[y.index.isin(train_index)], y[y.index.isin(test_index)]
#    
#     rf_opt_log.fit(X_train, np.log(y_train))
# 
#     y_pred = np.exp(rf_opt_log.predict(X_test))
# 
#     print("MSE по RandomForest:", mean_squared_error(y_test, y_pred))
#     print("R2 по RandomForest:", r2_score(y_test, y_pred))
#     rf_opt_mape_value = mape(y_test, y_pred)
#     rf_opt_mape_values.append(rf_opt_mape_value)
#     print("MAPE по RandomForest:",rf_opt_mape_value)
# 
# print(f"Среднее MAPE по RandomForest на выборке из 5 при стандартизированной целевой и гиперпараметрах: {(np.mean(rf_opt_mape_values) * 100):0.2f}%.")

Среднее MAPE по RandomForest на выборке из 5 при стандартизированной целевой и гиперпараметрах: 27.16%.

## 9.6. ExtraTreesRegressor

In [212]:
# with log-transformation of the target variable
etr_log = ExtraTreesRegressor(random_state=RANDOM_SEED, n_jobs=-1, verbose=1)

skf = KFold(n_splits=5, random_state=RANDOM_SEED)
etr_log_mape_values = []

for train_index, test_index in skf.split(X, y):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[X.index.isin(train_index)], X[X.index.isin(test_index)]
    y_train, y_test = y[y.index.isin(train_index)], y[y.index.isin(test_index)]
    
    etr_log.fit(X_train, np.log(y_train))
    y_pred = np.exp(etr_log.predict(X_test))

    print("MSE по ExtraTrees: ", mean_squared_error(y_test, y_pred))
    print("R2 по ExtraTrees: ", r2_score(y_test, y_pred))
    etr_log_mape_value = mape(y_test, y_pred)
    etr_log_mape_values.append(etr_log_mape_value)
    print("MAPE по ExtraTrees: ", etr_log_mape_value)

print(f"Среднее MAPE по ExtraTrees на выборке из 5 при стандартизированной целевой: {(np.mean(etr_log_mape_values) * 100):0.2f}%.")

Среднее MAPE по ExtraTrees на выборке из 5 при стандартизированной целевой: 12.78%.

ищем гиперпараметры и проверяем новую модель

In [213]:
# hp.uniform('n_estimators',100,500),
# hp.choice("n_estimators", [int(x) for x in np.linspace(200, 1000, num = 17)])

# def objective(params):
#     model=ExtraTreesRegressor(
#         n_estimators=int(params['n_estimators']),
#         max_depth=int(params['max_depth']),
#         min_samples_leaf=int(params['min_samples_leaf']),
#         min_samples_split=int(params['min_samples_split']),
#         bootstrap=params['bootstrap'],
#         max_features=params['max_features'],
#         random_state=RANDOM_SEED,
#         n_jobs=-1
#     )
#     model.fit(X_train, np.log(y_train))
#     pred=model.predict(X_test)
#     score=mape(y_test,np.exp(pred))
#     return score

# def optimize(trial):
#     params={
#         'n_estimators': hp.uniform('n_estimators',100,500),
#         'max_features': hp.choice("max_features", ['auto', 'sqrt']),
#         'max_depth': hp.uniform('max_depth',5,15),
#         'min_samples_split': hp.uniform('min_samples_split',2,10),
#         'min_samples_leaf': hp.uniform('min_samples_leaf',1,5),
#         'bootstrap': hp.choice("bootstrap", [True, False])
#     }
#     best=fmin(fn=objective, space=params, algo=tpe.suggest, trials=trial, max_evals=100, rstate=np.random.RandomState(RANDOM_SEED))
#     return best

# trial=Trials()
# best=optimize(trial)
# print(best)

100%|██████████| 100/100 [18:35<00:00, 11.16s/trial, best loss: 0.10470771469399263]{'bootstrap': 0, 'max_depth': 14.423984944704086, 'max_features': 0, 'min_samples_leaf': 2.642797554128248, 'min_samples_split': 3.9862256407095016, 'n_estimators': 147.78576378386654}


In [214]:
# etr_log_opt = ExtraTreesRegressor(random_state=RANDOM_SEED, 
#                                    n_jobs=-1, 
#                                    verbose=1, 
#                                    n_estimators = 148, 
#                                    min_samples_split = 4, 
#                                    min_samples_leaf = 3, 
#                                    max_features = 1, 
#                                    max_depth = 14, 
#                                    bootstrap = 0)
# 
# skf = KFold(n_splits=5, random_state=RANDOM_SEED)
# etr_log_opt_mape_values = []
# 
# for train_index, test_index in skf.split(X, y):
#     print("TRAIN:", train_index, "TEST:", test_index)
#     X_train, X_test = X[X.index.isin(train_index)], X[X.index.isin(test_index)]
#     y_train, y_test = y[y.index.isin(train_index)], y[y.index.isin(test_index)]
#     
#     etr_log_opt.fit(X_train, np.log(y_train))
#     y_pred = np.exp(etr_log_opt.predict(X_test))
# 
#     print("MSE по ExtraTrees: ", mean_squared_error(y_test, y_pred))
#     print("R2 по ExtraTrees: ", r2_score(y_test, y_pred))
#     etr_log_opt_mape_value = mape(y_test, y_pred)
#     etr_log_opt_mape_values.append(etr_log_opt_mape_value)
#     print("MAPE по ExtraTrees: ", etr_log_opt_mape_value)
# 
# print(f"Среднее MAPE по ExtraTrees на выборке из 5 при стандартизированной целевой и гиперпараметрах: {(np.mean(etr_log_opt_mape_values) * 100):0.2f}%.")

Среднее MAPE по ExtraTrees на выборке из 5 при стандартизированной целевой и гиперпараметрах: 52.29%.

## 9.7. XGBoostRegressor

In [215]:
# xgb_log = xgb.XGBRegressor(
#     objective='reg:squarederror', 
#     colsample_bytree=0.5,               
#     learning_rate=0.05, 
#     max_depth=12, 
#     alpha=1,                   
#     n_estimators=1000,
#     random_state=RANDOM_SEED,
#     verbose=1, 
#     n_jobs=-1,
# )
# 
# skf = KFold(n_splits=5, random_state=RANDOM_SEED)
# xgb_log_mape_values = []
# 
# for train_index, test_index in skf.split(X, y):
#     print("TRAIN:", train_index, "TEST:", test_index)
#     X_train, X_test = X[X.index.isin(train_index)], X[X.index.isin(test_index)]
#    y_train, y_test = y[y.index.isin(train_index)], y[y.index.isin(test_index)]
#    
#    xgb_log.fit(X_train, np.log(y_train))
#    y_pred = np.exp(xgb_log.predict(X_test))
# 
#     print("MSE по XGBoostRegressor: ", mean_squared_error(y_test, y_pred))
#     print("R2 по XGBoostRegressor: ", r2_score(y_test, y_pred))
#     xgb_log_mape_value = mape(y_test, y_pred)
#     xgb_log_mape_values.append(xgb_log_mape_value)
#     print("MAPE по XGBoostRegressor: ", xgb_log_mape_value)
# 
# print(f"Среднее MAPE по XGBoostRegressor на выборке из 5 при стандартизированной целевой: {(np.mean(xgb_log_mape_values) * 100):0.2f}%.")

The MAPE metric in the notebook is: 14.78%, on the leaderboard: 10.75%. 

## 9.8. StackingRegressor

In [216]:
# estimators = [
#     ('etr', ExtraTreesRegressor(random_state=RANDOM_SEED, n_jobs=-1, verbose=1)),
#     ('xgb', xgb.XGBRegressor(objective='reg:squarederror', colsample_bytree=0.5, learning_rate=0.05, max_depth=12, alpha=1, n_jobs=-1, n_estimators=1000, random_state=RANDOM_SEED))
# ]
# 
# sr_log = StackingRegressor(
#     estimators=estimators,
#     final_estimator=RandomForestRegressor()
# )
# 
# 
# sr_log.fit(X_train, np.log(y_train))
# y_pred = np.exp(sr_log.predict(X_test))
# 
# print(f"MAPE по StackingRegressor: {(mape(y_test, y_pred) * 100):0.2f}%.")

MAPE по StackingRegressor: 15.17%.

## 9.9. Результат:
Точность наивной модели по MAPE: 20.07%.

MAPE по ЛР: 70.03186281288252%

Среднее MAPE по CatBoost на выборке из 5 при стандартизированной целевой: 14.24%.

Среднее MAPE по RandomForest на выборке из 5 при стандартизированной целевой: 13.66%.

Среднее MAPE по ExtraTrees на выборке из 5 при стандартизированной целевой: 12.78%.

Среднее MAPE по XGBoostRegressor на выборке из 5 при стандартизированной целевой: 13.44%.

MAPE по StackingRegressor: 15.17%.


# 10. Submission

In [217]:
X.columns, data.columns

In [218]:
X_sub = data.query('markup == 0').drop(['price', 'markup', 'parsing_date', 'price_log2', 'price'], axis=1)

In [219]:
X_sub.info(), df_sub.info()

In [220]:
VERSION=13
predict_submission = np.exp(etr_log.predict(X_sub))
df_sub['price'] = predict_submission
df_sub.to_csv(f'submission.csv', index=False)
df_sub.head(10)